## Проект IT департамента нефтяной компании

Описание проекта

В IT департаменте нефтедобывающей компании нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов.

Шаги для выбора локации:

* В избранном регионе ищут месторождения, для каждого определяют значения признаков;
* Строят модель и оценивают объём запасов;
* Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
* Прибыль равна суммарной прибыли отобранных месторождений

Описание данных

* id — уникальный идентификатор скважины;
* f0, f1, f2 — три признака точек (возможно хим состав, наличие воды  и примесей, важные признаки);
* product — объём запасов в скважине (тыс. баррелей).

**Задача**:

Построить модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализировать возможную прибыль и риски техникой Bootstrap.

## 1. Загрузка и подготовка данных

In [1]:
import pandas as pd
import numpy as np

from scipy import stats as st
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [2]:
RND = np.random.RandomState(12345) #рэндом стейт
BOOTHSTR_SAMPLS = 1000 #шаги бутстрепа
BUDGET = 10**10 #бюджет на разработку скважин, руб 
PER_1K_BARRELL = 450000 #Доход с каждой единицы продукта, руб
N_SAMPLS = 500 #исследуемые точки для разработки
WELLS_NUMBER = 200 #количество лучших точек для разработки

In [4]:
reg0 = pd.read_csv('data_0.csv')
reg1 = pd.read_csv('data_1.csv')
reg2 = pd.read_csv('data_2.csv')
regs = [reg0, reg1, reg2]

In [5]:
reg0.head()

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


In [6]:
reg1.head()

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


In [7]:
reg2.head()

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


In [8]:
for reg in regs:
    print(reg.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column  

In [9]:
for reg in regs:
    print(reg.duplicated().sum())

0
0
0


In [10]:
for reg in regs:
    print(len(reg['id'].unique()))

99990
99996
99996


In [11]:
for i in range(3):
    regs[i] = regs[i].drop_duplicates(subset=['id']).reset_index(drop=True)
    regs[i] = regs[i].drop(['id'], axis=1)
#оставим уникальные значения скважин

In [12]:
for reg in regs:
    print(reg.info())
#проверим, все ли ок

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99990 entries, 0 to 99989
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   f0       99990 non-null  float64
 1   f1       99990 non-null  float64
 2   f2       99990 non-null  float64
 3   product  99990 non-null  float64
dtypes: float64(4)
memory usage: 3.1 MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99996 entries, 0 to 99995
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   f0       99996 non-null  float64
 1   f1       99996 non-null  float64
 2   f2       99996 non-null  float64
 3   product  99996 non-null  float64
dtypes: float64(4)
memory usage: 3.1 MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99996 entries, 0 to 99995
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   f0       99996 non-null  float64
 1   f1       99

In [13]:
reg0.corr()

,f0,f1,f2,product
f0,1.000000,-0.440723,-0.003153,0.143536
f1,-0.440723,1.000000,0.001724,-0.192356
f2,-0.003153,0.001724,1.000000,0.483663
product,0.143536,-0.192356,0.483663,1.000000


В Регионе 0 видна корреляция 48% между Product и F2, а также обратная корреляция 44% между признаками F0 и F1

In [14]:
reg1.corr()

,f0,f1,f2,product
f0,1.000000,0.182287,-0.001777,-0.030491
f1,0.182287,1.000000,-0.002595,-0.010155
f2,-0.001777,-0.002595,1.000000,0.999397
product,-0.030491,-0.010155,0.999397,1.000000


В Регионе 1 почти 100-процентная (!) коррелляция у Product и F2

In [15]:
reg2.corr()

,f0,f1,f2,product
f0,1.000000,0.000528,-0.000448,-0.001987
f1,0.000528,1.000000,0.000779,-0.001012
f2,-0.000448,0.000779,1.000000,0.445871
product,-0.001987,-0.001012,0.445871,1.000000


В Регионе 2 корреляция 44% между Product и F2

## 2. Обучение и проверка модели

In [16]:
def make_predictictions(data):
    features = data.drop(['id', 'product'], axis = 1)
    target = data['product']
    features_train, features_valid, target_train, target_valid = train_test_split(features, target, test_size=.25, random_state=RND)
    model = LinearRegression()
    model.fit(features_train, target_train)
    return pd.Series(model.predict(features_valid)), target_valid

In [17]:
predicted_1, target_1 = make_predictictions(reg0)
predicted_2, target_2 = make_predictictions(reg1)
predicted_3, target_3 = make_predictictions(reg2)

In [18]:
def show_rmse(answers, predictions):
    mse = mean_squared_error(answers, predictions)
    print('Средний запас сырья, тыс. баррелей:', predictions.mean())
    print('RMSE:', mse ** 0.5)

In [19]:
print('Регион 1')
show_rmse(target_1, predicted_1)

Регион 1
Средний запас сырья, тыс. баррелей: 92.59256778438008
RMSE: 37.5794217150813


In [20]:
print('Регион 2')
show_rmse(target_2, predicted_2)

Регион 2
Средний запас сырья, тыс. баррелей: 68.7699514579978
RMSE: 0.889736773768064


In [21]:
print('Регион 3')
show_rmse(target_3, predicted_3)

Регион 3
Средний запас сырья, тыс. баррелей: 95.08752812252328
RMSE: 39.958042459521614


Самое маленькое значение RMSE вышло у Региона 2 - 0.89

## 3. Подготовка к расчёту прибыли

In [22]:
BUDGET_FOR_ONE = BUDGET / WELLS_NUMBER
Min_mean_product = BUDGET_FOR_ONE / PER_1K_BARRELL
Min_mean_product

111.11111111111111

Минимальный объем сырья при котором разработка месторождения не будет убыточной

In [23]:
print("Средний истинный запас сырья в регионах, тыс. баррелей:")
count = 0
targets = [target_1, target_2, target_3]
for target in targets:
    count += 1
    print('Регион ', count,': ',  target.mean())

Средний истинный запас сырья в регионах, тыс. баррелей:
Регион  1 :  92.07859674082941
Регион  2 :  68.77162424986196
Регион  3 :  94.74895871720257


Исследуя средний истинный запас сырья в регионах, разработка месторождений в любом регионе будет убыточной.

Выберем 500 скважин из которых отберем 200 лучших

In [24]:
def revenue(target, probabilities):
    target = pd.Series(target).reset_index(drop=True)
    probabilities = pd.Series(probabilities).reset_index(drop=True)
    probs_sorted = probabilities.sort_values(ascending=False)
    selected = target[probs_sorted.index][:WELLS_NUMBER]
    mlrd = 1000000000 #чтобы округлить до миллиарда
    return round((((PER_1K_BARRELL * selected.sum()) - BUDGET) / mlrd),2)

In [25]:
revenue(target_1, predicted_1)

3.32

In [26]:
revenue(target_2, predicted_2)

2.42

In [27]:
revenue(target_3, predicted_3)

2.54

**Вывод**: Самый прибыльный регион - Регион 1, так как может принести прибыль 3.32 миллиардов рублей

## 4. Расчёт прибылей и рисков

In [28]:
def bootstrap(target, predictions):
    values = []
    for i in range(BOOTHSTR_SAMPLS):
        target_subsample = target.reset_index(drop=True).sample(n=N_SAMPLS, replace=True, random_state=RND)
        probs_subsample = predictions[target_subsample.index]
        rev = revenue(target_subsample, probs_subsample)
        values.append(rev)
    risk = 0.025
    values = pd.Series(values)
    lower = values.quantile(risk)
    confidence_interval = (values.quantile(0.025), values.quantile(0.975))
    mean = values.mean()

    print("Средняя выручка:", mean)
    print("2.5%-квантиль:", lower)
    print("Доверительный интервал:", confidence_interval)
    print("Риск убытка:", (values < 0).mean())

In [29]:
bootstrap(target_1, predicted_1)

Средняя выручка: 0.39424000000000015
2.5%-квантиль: -0.07024999999999999
Доверительный интервал: (-0.07024999999999999, 0.92)
Риск убытка: 0.056


In [30]:
bootstrap(target_2, predicted_2)

Средняя выручка: 0.45471
2.5%-квантиль: 0.06
Доверительный интервал: (0.06, 0.86)
Риск убытка: 0.007


In [31]:
bootstrap(target_3, predicted_3)

Средняя выручка: 0.3536300000000003
2.5%-квантиль: -0.16024999999999998
Доверительный интервал: (-0.16024999999999998, 0.85)
Риск убытка: 0.071


## Общий вывод

Выберем Регион 2, т.к. средняя выручка самая большая и риск убытка самый маленький среди трех регионов, а также самый низкая RMSE.